In [1]:
import numpy as np
import pygame
from pygame.locals import (
    K_UP,
    K_DOWN,
    K_LEFT,
    K_RIGHT,
    K_ESCAPE,
    KEYDOWN,
    QUIT,
)
pygame.init()

pygame 2.6.0 (SDL 2.28.4, Python 3.12.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


(5, 0)

In [15]:
class Node(pygame.sprite.Sprite):

    COLOR_INIT = (34, 227, 147)
    COLOR_CAPTURED = (151, 23, 255)
    WHITE = (255,255,255)
    DIST = 20
    Kp = 0.1
    Kd = Kp/3.
    BUFFER_SIZE = 20
    R=10

    def __init__(self, x=0, y=0,
                 coef=0.9):
        super().__init__()
        self.surf = pygame.Surface([Node.R*2, Node.R*2])
        self.surf.fill(Node.WHITE)
        self.surf.set_colorkey(Node.WHITE)
        pygame.draw.circle(surface=self.surf,
            color=self.COLOR_INIT,center=(Node.R,Node.R),
            radius=Node.R)
        
        self.rect = self.surf.get_rect()
        self.rect.x = x-Node.R
        self.rect.y = y-Node.R
        self.past_error = 0.0
        self.next = None
        self.buffer = []
        self.target_x = None
        self.target_y = None
        self.history_coef = coef

    @property
    def center(self):
        return self.rect.x+Node.R, self.rect.y+Node.R

    def update(self,x,y, head=True):
        
        if head:
            self._update_head(x,y)
        else:
            self._update(x,y)

    def set_color(self, color="captured"):
        c = Node.COLOR_CAPTURED if color=="captured" else Node.COLOR_INIT
        pygame.draw.circle(surface=self.surf,
                           color=c,center=(Node.R,Node.R),
                            radius=Node.R)
        
    def _update_head(self, x, y):
        cx,cy = self.center
        if self.target_x is None:
            self.target_x = x
            self.target_y = y
        else:
            self.target_x = self.history_coef*self.target_x + \
                (1-self.history_coef)*x
            self.target_y = self.history_coef*self.target_y + \
                (1-self.history_coef)*y
        x = self.target_x
        y = self.target_y
        
        d = np.sqrt((cx-x)**2 + (cy-y)**2)
        e = np.abs(d-self.DIST)
        de = e-self.past_error
        self.past_error = e
        e_ = (e*Node.Kp + de*Node.Kd)
        dx = (x-cx)/d*e_
        dy = (y-cy)/d*e_
        self.rect.x += dx
        self.rect.y += dy
    
    def _update(self, x, y):
        self.rect.x = x-Node.R
        self.rect.y = y-Node.R

def binary_search(arr, x, l=0):
    """binary search; can supply starting index with l

    Args:
        arr (_type_): _description_
        x (_type_): _description_
        l (int, optional): _description_. Defaults to 0.

    Returns:
        _type_: _description_
    """
    l = l if l else 0
    r = len(arr)-1
    while l <= r:
        mid = l + (r-l)//2
        if arr[mid] == x:
            return mid
        elif arr[mid] < x:
            l = mid + 1
        else:
            r = mid - 1
    return l

class Snake():
    def __init__(self,init_node,
                 capture_dist=80) -> None:
        self.nodes = pygame.sprite.Group()
        self.nodes.add(init_node)
        self.head = init_node
        self.head_traj = [list(init_node.center)]
        self.traj_cum_len = [0.0]
        
        self.seg_len = 2.1*Node.R # length of each segment
        self.capture_dist = self.seg_len
        self.n_nodes = 1

    def update(self,x,y):
        # update head trajectory
        cx,cy = self.head.center
        px,py = self.head_traj[0]
        self.head_traj.insert(0, [cx,cy])

        d = np.sqrt((cx-px)**2 + (cy-py)**2)
        for i, _ in enumerate(self.traj_cum_len):
            self.traj_cum_len[i] += d
        self.traj_cum_len.insert(0, 0.0)

        while self.traj_cum_len[-1] > self.seg_len*(self.n_nodes+1):
            self.traj_cum_len.pop(-1)
            self.head_traj.pop(-1)
            if len(self.head_traj) == 0:
                break

        self.head.update(x,y,head=True)
        next = self.head.next
        ind = 1
        k_prev = None
        while next:
            next:Node
            sl = ind*self.seg_len
            k = binary_search(self.traj_cum_len, sl, k_prev)
            if k<=0 or k>=len(self.traj_cum_len):
                break
            k_prev = k
            # print(f"found {k}")
            # print(len(self.traj_cum_len),len(self.head_traj))
            rho = (sl-self.traj_cum_len[k-1]) / \
                (self.traj_cum_len[k]-self.traj_cum_len[k-1])
            x = (1-rho)*self.head_traj[k-1][0] + \
                rho*self.head_traj[k][0]
            y = (1-rho)*self.head_traj[k-1][1] + \
                rho*self.head_traj[k][1]
            next.update(x,y,head=False)
            next = next.next
            ind+=1

    def append_head(self, new_node):
        new_node.next = self.head
        self.head = new_node
        self.nodes.add(new_node)
    
    def capture(self, node : Node):
        """check if head is within some threshold of the rogue
        node

        Args:
            node (_type_): _description_
        """
        cx1,cy1 = self.head.center
        cx2,cy2 = node.center
        d = np.sqrt((cx1-cx2)**2 + (cy1-cy2)**2)
        if d < self.capture_dist:
            node.next = self.head
            self.head = node
            node.set_color("captured")
            self.nodes.add(node)
            self.n_nodes += 1
            # print("captured")
            return 1
        return 0

In [16]:
# Set up the drawing window
WIDTH = 500
HEIGHT = 500

def spawn_new_node():
    return Node(x=np.random.randint(Node.R,WIDTH-Node.R),
            y=np.random.randint(Node.R,HEIGHT-Node.R))
screen = pygame.display.set_mode([WIDTH, HEIGHT])
screen.fill((255, 255, 255))
clock = pygame.time.Clock()

running = True
init_node = Node(100,100)
init_node.set_color("captured")
snake = Snake(init_node)
# node2 = Node(x=85,y=100)
# snake.append_head(node2)
all_nodes = pygame.sprite.Group()
all_nodes.add(init_node)
# spawn new node
new_node = spawn_new_node()
all_nodes.add(new_node)
# Main loop
spawn_counter = 1
capture_counter = 0
ignore_counts = 10
ignore_counter = 0
while running:
    # Look at every event in the queue
    for event in pygame.event.get():
        # Did the user hit a key?
        if event.type == KEYDOWN:
            # Was it the Escape key? If so  , stop the loop.
            if event.key == K_ESCAPE:
                running = False

        # Did the user click the window close button? If so, stop the loop.
        elif event.type == QUIT:
            running = False

    if snake.capture(new_node):
        capture_counter += 1
    
    if capture_counter >= spawn_counter:
        new_node = spawn_new_node()
        all_nodes.add(new_node)
        spawn_counter += 1

    x, y = pygame.mouse.get_pos()
    snake.update(x,y)
    
    screen.fill((255, 255, 255))
    for node in all_nodes:
        screen.blit(node.surf, node.rect)

    pygame.display.update()

    clock.tick(30)

pygame.quit()

In [54]:
# Import and initialize the pygame library
import pygame
pygame.init()

# Set up the drawing window
screen = pygame.display.set_mode([500, 500])

# Run until the user asks to quit
running = True
while running:

    # Did the user click the window close button?
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Fill the background with white
    screen.fill((255, 255, 255))

    # Draw a solid blue circle in the center
    pygame.draw.circle(screen, (0, 0, 255), (250, 250), 75)

    # Flip the display
    pygame.display.flip()

# Done! Time to quit.
pygame.quit()